In [20]:
import pandas as pd
import altair as alt

# Load data
data = pd.read_csv("NBA_Players.csv")

# Convert columns to numeric for metrics
data['pts'] = pd.to_numeric(data['pts'], errors='coerce')
data['reb'] = pd.to_numeric(data['reb'], errors='coerce')
data['ast'] = pd.to_numeric(data['ast'], errors='coerce')

# Ensure 'season' is treated as a categorical type
data['season'] = data['season'].astype(str)

# Dropdown selector
season_dropdown = alt.binding_select(options=sorted(data['season'].unique()), name='Select Season: ')
season_selection = alt.param(
    name="season_selection", 
    bind=season_dropdown, 
    value=sorted(data['season'].unique())[0]
)

# Function to create top 10 chart for a given metric
def create_top10_chart(metric, title, color):
    filtered = data[data[metric].notnull()]  # Filter rows with non-null values for the metric
    chart = (
        alt.Chart(filtered)
        .mark_bar()
        .encode(
            x=alt.X(metric, title=f"Top 10 {title}"),
            y=alt.Y("player_name:N", sort='-x', title="Player"),
            color=alt.value(color),
            tooltip=[
                alt.Tooltip("player_name:N", title="Player"),
                alt.Tooltip("season:N", title="Season"),
                alt.Tooltip(metric, title=title),
            ],
        )
        .transform_filter(f"season_selection == datum.season")
        .transform_window(
            rank=f"rank({metric})",
            sort=[alt.SortField(metric, order="descending")],
            groupby=["season"],
        )
        .transform_filter(alt.datum.rank <= 10)
        .properties(
            width=400,
            height=300,
            title=f"Top 10 Players by {title}",
        )
    )
    return chart

# Create individual charts
points_chart = create_top10_chart("pts", "Points", "steelblue")
rebounds_chart = create_top10_chart("reb", "Rebounds", "orange")
assists_chart = create_top10_chart("ast", "Assists", "green")

# Combine charts into a single view
combined_chart = alt.vconcat(
    points_chart, rebounds_chart, assists_chart
).add_params(season_selection)

# Save chart to an HTML file
combined_chart.save("top10_players_by_season.html")
